In [ ]:
import numpy as np
import pandas as pd
import spotipy
import yaml
from spotipy.oauth2 import SpotifyClientCredentials


## Download data from Spotify API

In [ ]:
# Read the credentials from credentials.yml
with open("credentials.yml") as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

# Set up the Spotify API
client_id = credentials["client_id"]  # YOUR_CLIENT_ID
client_secret = credentials["client_secret"]  # YOUR_CLIENT_SECRET
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
